<a href="https://colab.research.google.com/github/RiccTrev/dimensionatore/blob/main/NON_TOCCARE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bilanciamento CER

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#!ls
#%cd

Mounted at /content/drive/


In [ ]:
#Import del modulo delle funzioni
import sys
import json
import requests

In [ ]:
#Import delle librerie
import datetime
import psycopg2
import pandas.io.sql as sqlio
import numpy as np
import pandas as pd
import os.path
from pathlib import Path
!pip install --upgrade openpyxl
!pip install numpy-financial
import numpy_financial as npf

#Print all rows of the tables
pd.set_option('display.max_rows', None)
#Precisione delle cifre
np.set_printoptions(precision=4)
#Imposta numero di cifre significative e separatore migliaia
pd.set_option('display.float_format','{:,.4}'.format)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title Funzioni
#Sample di chiamata di funzioni per riga: df_finale['Esempio'] = df.apply(funzioni.NomeFunzione, axis = 1))
def CalcolaAutoconsumo(df):
  if df['Immissione'] > 0: 
    Autoconsumo = 0
    return Autoconsumo
  else:
    if df['Fascia 1'] + 0.2 * df['Fascia 2'] > df['Produzione Aggiuntiva'] or df['Fascia 1'] > df['Produzione Aggiuntiva']:
      Autoconsumo = df['Produzione Aggiuntiva']
      return Autoconsumo
    if df['Fascia 2'] >= 1/3 * df['Fascia 1']:  
      Autoconsumo = df['Fascia 1'] + 0.2 * df['Fascia 2']
      return Autoconsumo
    else:
      Autoconsumo = df['Fascia 1']
      return Autoconsumo
  
def CalcolaImmissione(df):
  if df['Immissione'] > 0: 
    Immissione = df['Immissione']
    return Immissione;
  else:
    if df['Fascia 2'] >= 1/3 * df['Fascia 1']:  
      Immissione = df['Produzione Aggiuntiva'] - df['Autoconsumo']
      if Immissione < 0: 
        raise TypeError('Attenzione! Immissione < 0 per l\'utente '+str(df['Identificativo']+'nel mese di '+str(df['Periodo']))) 
      return Immissione
    else:
      Immissione = df['Produzione Aggiuntiva'] - df['Autoconsumo']
      if Immissione < 0: 
        raise TypeError('Attenzione! Immissione < 0 per l\'utente '+str(df['Identificativo']+' nel mese di '+str(df['Periodo']))) 
      return Immissione
  

def VerificaIdro(df): 
    if 'IDRO' in df['Identificativo']:
      return 1
    else: 
      return 0

def StimaProduzione(df):
  mesi = {
      0:'gen',
      1:'feb',
      2:'mar',
      3:'apr',
      4:'mag',
      5:'giu',
      6:'lug',
      7:'ago',
      8:'set',
      9:'ott', 
      10:'nov', 
      11:'dic'
  }
  output = pd.DataFrame(columns = ['Identificativo', 'Lat', 'Long', 'Periodo', 'Produzione'])
  old_id = str(0)
  for index, row in df.iterrows(): 
    if row['Potenza Da Installare'] == 0: 
      continue
    if old_id == row['Identificativo']: 
      continue
    else: 
      old_id = row['Identificativo']
    response = requests.get("https://re.jrc.ec.europa.eu/api/v5_1/PVcalc?lat="+str(row['Lat'])+"&lon="+str(row['Long'])+"&raddatabase=PVGIS-SARAH&peakpower="+str(row['Potenza Da Installare'])+"&pvtechchoice=crystSi&loss="+str(16)+"&optimalinclination=1&vertical_optimum=1&outputformat=json")
    todos = json.loads(response.text)
    #for i in range (1,13)
    for i in range (0,12): 
      new_row = {}
      new_row = {'Identificativo': row['Identificativo'], 'Lat':row['Lat'], 'Long':row['Long'],'Periodo':mesi[i], 'Produzione': todos['outputs']['monthly']['fixed'][i]['E_m']}
      output = output.append(new_row, ignore_index=True)
  return output

#1) Aprire file in input


In [ ]:
#MODIFICARE QUA IL NOME DELLA DIRECTORY DI LAVORO
directory = '/content/drive/MyDrive/Simulazioni/1-Temporary'

In [ ]:
#Percorso del file revised creato con il notebook 'Preparazione'
try:
  fn = directory+'/Sample_DER.xlsx'
  xl = pd.ExcelFile(fn)
  xl.sheet_names
except: 
  raise TypeError('Non sono riuscito ad aprire il file.') 

# 2) Costo Energia - RID - Incentivo condivisione

In [ ]:
#COSTO MEDIO ENERGIA (€/kWh): 
costo_medio_energia = 0.25
#INCENTIVO ARERA (energia condivsa) - restituzione costi di sistema (€/kWh)
incentivo_arera= 0.009
#INCENTIVO MISE (energia condivisa) - incentivazione energia condivisa (€/kWh)
incentivo_mise = 0.110
#RITIRO DEDICATO - valorizzazione energia immessa (€/kWh)
ritiro_dedicato = 0.090

###IDRO
#Valorizzazione energia prodotta idro (€/kWh)
val_prod_idro = 0.090
#Euro ricevuti dalla condivisione per l'energia (€/kWh)
inc_cond_idro = 0.070

Organizzo i fogli in un dizionario di DataFrame

In [ ]:
#Creo un dizionario di DataFrame. 
dfs = {sh:xl.parse(sh, header = 0) for sh in xl.sheet_names}
#Stampo le chiavi del dizionario (nomi dei workspace di excel)
dfs.keys()

dict_keys(['CER', 'SSP'])

In [ ]:
#DEBUG
#Stampo per visualizzare i data frame: analisi visiva che tutto sia ok
#RIMUOVERE COMMENTO SE SI VUOLE FAR GIRARE

# for key in dfs.keys():
#     print(key)
#     print(dfs[key])


In [ ]:
dft = dfs['CER']
dft['Fascia 1'].sum()*7/5

292430.04

In [ ]:
df = dfs['CER']

Scarico dati di produzione e calcolo autoconsumo ed immissione per nuovi prosumer

In [ ]:
df_produzione = StimaProduzione(df)

In [ ]:
df = pd.merge(df, df_produzione, on=('Identificativo', 'Periodo'), how = 'left')
df = df.drop(columns = ['Lat_y', 'Long_y', 'Lat_x', 'Long_x'])
df = df.rename(columns={'Produzione': 'Produzione Aggiuntiva'})
df = df.fillna(0)

In [ ]:
#Calcolo autoconsumo ed immissione per gli utenti CONSUMER. N.B. Non è possible calcolarlo per gli utenti prosumer poiché non abbiamo il consumo totale, ma solo il prelevamento dalla rete.
df['Autoconsumo'] = df.apply(CalcolaAutoconsumo, axis = 1)
df['Immissione'] = df.apply(CalcolaImmissione, axis = 1)

In [ ]:
df

,Identificativo,Periodo,Fascia 1,Fascia 2,Fascia 3,Immissione,Bolletta Mensile,Potenza Da Installare,Produzione Aggiuntiva,Autoconsumo
0,A,gen,8.529e+03,4.578e+03,5.451e+03,328.0,6.255e+03,16.0,1.538e+03,0.0
1,A,feb,9.596e+03,4.523e+03,4.822e+03,297.0,6.384e+03,16.0,1.637e+03,0.0
2,A,mar,9.086e+03,4.324e+03,4.44e+03,319.0,6.017e+03,16.0,2.08e+03,0.0
3,A,apr,9.88e+03,4.829e+03,5.853e+03,297.0,6.931e+03,16.0,2.235e+03,0.0
4,A,mag,1.093e+04,5.711e+03,7.389e+03,319.0,1.105e+04,16.0,2.458e+03,0.0
5,A,giu,1.285e+04,6.729e+03,9.008e+03,438.0,1.314e+04,16.0,2.536e+03,0.0
6,A,lug,1.322e+04,7.826e+03,9.812e+03,755.0,1.419e+04,16.0,2.657e+03,0.0
7,A,ago,8.522e+03,5.799e+03,9.822e+03,13.0,1.11e+04,16.0,2.629e+03,0.0
8,A,set,6.3e+03,4.972e+03,8.105e+03,40.0,6.531e+03,16.0,2.256e+03,0.0
9,A,ott,6.987e+03,4.733e+03,7.153e+03,129.0,6.362e+03,16.0,2.039e+03,0.0


In [ ]:
df_produzione['Produzione'].sum()/12

258835.31916666662

In [ ]:
#df = dfs['CER']
df_ssp = dfs['SSP']

#df = df.drop(columns=['Bolletta Annua'])

temporary = df.groupby(by=["Identificativo"], as_index=True).agg({"Fascia 1":"sum", "Fascia 2":"sum", 'Immissione':'sum', 'Autoconsumo':'sum', 'Produzione Aggiuntiva':'sum'}).rename(columns={"Fascia 1":"Somma Fascia 1", "Fascia 2":"Somma Fascia 2", 'Immissione': 'Somma Immissione', 'Autoconsumo': 'Somma Autoconsumo', 'Produzione Aggiuntiva':'Somma Produzione Aggiuntiva'}).reset_index()
df = pd.merge(df, temporary, on=['Identificativo'], how="left")
df = df.drop(columns = ['Autoconsumo' , 'Produzione Aggiuntiva'])
df['Consumo Per Fotovoltaico'] = df.apply(lambda x: (x['Somma Fascia 1']) if (x['Somma Fascia 2'] < 0.5*x['Somma Fascia 1'])  else (x['Somma Fascia 1']*12/10), axis =1) 
df['Consumo Per Fotovoltaico'] = df['Consumo Per Fotovoltaico'] - df['Somma Autoconsumo'] 
#df['Produzione'] = df['Taglia Impianto'] * 1250
df['Differenza'] = df.apply(lambda x: (1/3*x['Somma Immissione'] - x['Consumo Per Fotovoltaico']) if (x['Identificativo'] == 'Idroelettrico') else (x['Somma Immissione'] - x['Consumo Per Fotovoltaico']), axis = 1)  # Questo è quanto prelevo/immetto al netto della produzione dell'impianto preesistente
#df['Differenza'] = df.apply(lambda x: df['Somma Immissione'] - df['Consumo Per Fotovoltaico'] # Questo è quanto prelevo/immetto al netto della produzione dell'impianto preesistente

df["Taglia Impianto Potenziale"] = df.apply(lambda x: (abs(x["Differenza"])/1250) if (x["Differenza"] < 0)  else (-x['Differenza']/1250) , axis =1)
df['Costo Impianto Potenziale'] = df.apply(lambda x: (x['Taglia Impianto Potenziale']*1200) if (x["Differenza"] < 0)  else (0) , axis =1) 
df['Costo Potenza da Installare'] = df.apply(lambda x: (x['Potenza Da Installare']*1200) if (x["Potenza Da Installare"] > 0)  else (0) , axis =1) 
df['Consumo Totale Fasce'] = (df['Fascia 1'] + df['Fascia 2'] + df['Fascia 3'])

In [ ]:
df['Taglia Impianto Potenziale'].sum()

-27223.686751999994

## Calcolo benefici SSP

In [ ]:
#Parametri economici SSP
PUN = 0.1
prezzo_energia = 0.9*PUN
corr_forfettario = 0.6*PUN

In [ ]:
def CalcolaBenficiSSP(df): 
  temporary_ssp = df.groupby(by=["Identificativo"], as_index=True).agg({"Produzione":"sum", "Autoconsumo":"sum", "Esportata":"sum", "Importata":"sum" }).rename(columns={"Produzione":"Totale Produzione", "Autoconsumo":"Totale Autoconsumo", "Esportata":"Totale Esportata", "Importata":"Totale Importata"}).reset_index()
  temporary_ssp['Beneficio SSP'] = temporary_ssp.apply(lambda x: min(PUN*x['Totale Importata'], prezzo_energia*x['Totale Esportata']) + corr_forfettario*min(x['Totale Importata'], x['Totale Esportata']), axis =1) 
  return temporary_ssp

In [ ]:
#Input
if df_ssp.empty: 
  print('Non sono stati istanziati utenti con scambio sul posto')
else: 
  print('Sono stati istanziati utenti con scambio sul posto')
  df_ssp = CalcolaBenficiSSP(df_ssp)
df_ssp
  

Sono stati istanziati utenti con scambio sul posto


,Identificativo,Totale Produzione,Totale Autoconsumo,Totale Esportata,Totale Importata,Beneficio SSP
0,Ciemme srl,1.897e+04,1.313e+04,7.18e+03,1.901e+04,1.077e+03
1,Latteria del Molise Srl,4.989e+04,3.715e+04,4.129e+03,4.67e+04,619.4


In [ ]:
#df

In [ ]:
#Inserisco consumo totale 
temporary2 = df.groupby(by=["Identificativo"], as_index=True).agg({"Consumo Totale Fasce":"sum"}).rename(columns={"Consumo Totale Fasce":"Consumo Totale Annuo"}).reset_index()
df = pd.merge(df, temporary2, on=['Identificativo'], how="left")

In [ ]:
#Inserisco costo bolletta totale
temporary3 =  df.groupby(by=["Identificativo"], as_index=True).agg({"Bolletta Mensile":"sum"}).rename(columns={"Bolletta Mensile":"Costo Bolletta"}).reset_index()
df = pd.merge(df, temporary3, on=['Identificativo'], how="left")

In [ ]:
#df

In [ ]:
df_finale = df.groupby('Identificativo').mean().drop(columns=['Fascia 1', 'Fascia 2', 'Fascia 3', 'Consumo Totale Fasce', 'Bolletta Mensile', 'Immissione']).reset_index()
#df_finale['Costo Bolletta'] = df_finale.apply(lambda x: ((x['Consumo Totale Annuo'] - x['Produzione'])*costo_medio_energia) if (x['Differenza']< 0) else (0), axis = 1)
#####################################################################################
#df_finale.loc[df_finale['Identificativo']=='Torrefazione', 'Costo Bolletta'] = 8971#
#####################################################################################
df_finale = df_finale.drop(columns=['Consumo Totale Annuo'])
df_finale

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,1.293e+05,1.92e+04,1.041e+05
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,0.0,2.4e+06,8.007e+03
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,0.0,0.0,1.041e+05


Variabile Taglia Impianto Potenziale

**Potenza da Installare** 

In [ ]:
df_finale['Taglia Impianto Potenziale'].sum()

-2268.640562666666

Produzione degli impianti potenziali

In [ ]:
df_finale['Taglia Impianto Potenziale'].sum()*1250

-2835800.7033333327

Energia Condivisa - Caso Impianto Plus

# 3) Specificare taglia impianto
Se taglia_specifica = 0 allora la simulazione viene condotta identificando la massima taglia necessaria per soddisfare un fabbisogno pari al consumo di F1 + il 40% di tale valore per ogni utenza istanziata. 
In formula: consumo per fotovoltaico = F1*(1+7/5)
Se viene specificata una taglia (in kWp) allora la simulazione viene condotta per la potenza specificata. 

---



In [ ]:
#Se taglia_specifica = -1 => viene calcolata la taglia ideale per soddisfare il fabbisogno di F1 + il 25% dei consumi di F1
taglia_specifica = -1

Inserisco riga produttore

In [ ]:
if taglia_specifica > 0: 
  df_finale['Taglia Impianto Potenziale'] = 0
  df_finale['Costo Impianto Potenziale'] = 0
  produttore = {'Identificativo':'Produttore', 'Taglia Impianto':taglia_specifica, 'Somma Fascia 1':0, 'Consumo Per Fotovoltaico':0, 'Somma Immissione':taglia_specifica*1250, 'Costo Bolletta': 0, 'Differenza':taglia_specifica*1250, 'Taglia Impianto Potenziale':taglia_specifica, 'Costo Impianto Potenziale': taglia_specifica*1200, 'Percentuale Condivisa': 0 }
  df_finale = df_finale.append(produttore, ignore_index=True)
elif taglia_specifica  == -1: 
  produttore = {'Identificativo':'Produttore', 'Taglia Impianto':df_finale['Taglia Impianto Potenziale'].sum(), 'Somma Fascia 1':0, 'Consumo Per Fotovoltaico':0, 'Somma Immissione':df_finale['Taglia Impianto Potenziale'].sum()*1250, 'Costo Bolletta': 0, 'Differenza':df_finale['Taglia Impianto Potenziale'].sum()*1250, 'Taglia Impianto Potenziale':df_finale['Taglia Impianto Potenziale'].sum(), 'Costo Impianto Potenziale': df_finale['Taglia Impianto Potenziale'].sum()*1200, 'Percentuale Condivisa': 0 }
  df_finale = df_finale.append(produttore, ignore_index=True)
elif taglia_specifica == 0: 
  df_finale['Taglia Impianto Potenziale'] = 0
  df_finale['Costo Impianto Potenziale'] = 0


Energia Condivisa - Caso Impianto Bilanciato


In [ ]:
if taglia_specifica == 0:
  print('non ho specificato la taglia')
  negativi = abs(df_finale[df_finale['Differenza'] <0]['Differenza'].sum()) #contributi in prelievo
  positivi = df_finale['Somma Immissione'].sum()
  condivisa = min(negativi, positivi)
  print('positivi vale:'+ str(positivi))
  print('negativi vale: ' +str(negativi))

else: 
  print('ho specificato la taglia')
  negativi = abs(df_finale[df_finale['Differenza'] <0]['Differenza'].sum()) #contributi in prelievo
  positivi = df_finale[(df_finale['Identificativo'] != 'Produttore') & (df_finale['Differenza'] > 0)]['Differenza'].sum() + df_finale[df_finale['Identificativo'] == 'Produttore']['Differenza'].sum() #contributi in immissione
  condivisa = min(negativi, positivi)
  print('positivi vale:'+ str(positivi))
  print('negativi vale: ' +str(negativi))

condivisa

ho specificato la taglia
positivi vale:134665.40000000037
negativi vale: 2970466.1033333326


134665.40000000037

In [ ]:
df_finale[df_finale['Differenza'] > 0]['Differenza'].sum()

2970466.103333333

In [ ]:
df_finale['Taglia Impianto Potenziale'].sum()*1250

-5671601.406666665

In [ ]:
print(negativi, positivi)

2970466.1033333326 134665.40000000037


In [ ]:
df_finale['Categoria'] = df_finale.apply(lambda x: ('Consumer') if (x["Somma Immissione"] == 0)  else ( 'Prosumer' if (x['Somma Immissione'] != 0 and x['Consumo Per Fotovoltaico'] != 0)  else 'Produttore') , axis =1) 
numero_produttori = len(df_finale[df_finale['Categoria'] == 'Produttore'])
numero_prosumer = len(df_finale[df_finale['Categoria'] == 'Prosumer'])
numero_partecipanti = df_finale['Identificativo'].count() - len(df_finale[df_finale['Identificativo'] == 'Idroelettrico']) # Partecipanti sono quelli che partecipano anche ai costi, l'idroelettrico non partecipa ai costi
numero_membri = df_finale['Identificativo'].count() #Membri sono tutti quelli che afferiscono alla CER
numero_consumer = numero_partecipanti - numero_prosumer - numero_produttori



In [ ]:
numero_partecipanti

3

In [ ]:
numero_membri

4

In [ ]:
numero_produttori

2

In [ ]:
numero_prosumer

2

In [ ]:
#Partecipazione in prelievo (tutti gli utenti)
tot_prelievo = abs(df_finale[df_finale['Differenza']<0]['Differenza'].sum())
df_finale['Percentuale Condivisa'] = df_finale.apply(lambda x: (abs(x['Differenza']) / tot_prelievo ) if (x["Differenza"] < 0)  else (0) , axis =1) 
#df_finale['Percentuale Condivisa'] = df_finale.apply(lambda x: (abs(x['Differenza'])/condivisa) if (x["Differenza"] < 0)  else (0) , axis =1) 


In [ ]:
df_finale

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,Percentuale Condivisa,Taglia Impianto,Categoria
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,1.293e+05,1.92e+04,1.041e+05,0.04533,NaN,Prosumer
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,0.0,2.4e+06,8.007e+03,0.0,NaN,Prosumer
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,0.0,0.0,1.041e+05,0.0,NaN,Produttore
3,Produttore,NaN,0.0,NaN,-2.836e+06,NaN,NaN,0.0,-2.836e+06,-2.269e+03,-2.722e+06,NaN,0.0,0.9547,-2.269e+03,Produttore


In [ ]:
#Partecipazione in immissione produttore & prosumer  df_finale[(df_finale['Identificativo'] != 'Idroelettrico') & (df_finale['Somma Immissione'] > 0)]
#df_finale[df_finale['Somma Immissione'] > 0]['Somma Immissione'].sum()
tot_immessa = df_finale[(df_finale['Identificativo'] != 'Idroelettrico') & (df_finale['Somma Immissione'] > 0)]['Somma Immissione'].sum()    # + df_finale[(df_finale['Differenza']>0) & (df_finale['Categoria'] != 'Produttore')]['Differenza'].sum()  #Immetto ciò che imposto nell'excel + eventule surplus della produzione nel momento in cui ho differenza > 0
df_finale['Percentuale Immessa'] = df_finale.apply(lambda x: (abs(x['Somma Immissione'])/tot_immessa) if (x["Somma Immissione"] > 0 and x['Identificativo'] != 'Idroelettrico')  else (0) , axis =1)

In [ ]:
tot_immessa

2972204.53

In [ ]:
#Partecipazione in immissione prosumer
tot_immessa_prosumer = df_finale[df_finale['Categoria'] == 'Prosumer']['Somma Immissione'].sum()
df_finale['Percentuale Immessa Prosumer'] = df_finale.apply(lambda x: (x['Somma Immissione']/tot_immessa_prosumer) if (x["Categoria"] == 'Prosumer')  else (0) , axis =1)

In [ ]:
df_finale
df_finale.fillna(0)

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,Percentuale Condivisa,Taglia Impianto,Categoria,Percentuale Immessa,Percentuale Immessa Prosumer
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,1.293e+05,1.92e+04,1.041e+05,0.04533,0.0,Prosumer,0.001389,0.001389
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,0.0,2.4e+06,8.007e+03,0.0,0.0,Prosumer,0.9986,0.9986
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,0.0,0.0,1.041e+05,0.0,0.0,Produttore,0.0,0.0
3,Produttore,0.0,0.0,0.0,-2.836e+06,0.0,0.0,0.0,-2.836e+06,-2.269e+03,-2.722e+06,0.0,0.0,0.9547,-2.269e+03,Produttore,0.0,0.0


Calcolo Partecipazione Condivisione

In [ ]:
Incentivo_Totale_Condivisione= condivisa*(incentivo_arera+incentivo_mise)
Incentivo_Totale_Condivisione

16025.182600000044

# 4) Ripartizione incentivo condivisione

In [ ]:
df_finale[df_finale['Identificativo'] == 'Idroelettrico']

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,Percentuale Condivisa,Taglia Impianto,Categoria,Percentuale Immessa,Percentuale Immessa Prosumer
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,0.0,0.0,1.041e+05,0.0,NaN,Produttore,0.0,0.0


In [ ]:
Incentivo_Totale_Condivisione

16025.182600000044

In [ ]:
Incentivo_Attribuibile = Incentivo_Totale_Condivisione
#Calcolo quota idroelettrico
if len(df_finale[df_finale['Identificativo'] == 'Idroelettrico']) > 0: #Se esiste un produttore idroelettrico aggiorna la quota attribuibile
  Quota_Idro = inc_cond_idro * df_finale[df_finale['Identificativo'] == 'Idroelettrico']['Somma Immissione'].sum() ##1/3 dell'energia che immette viene remunerata a 70€/MWh
  if(Quota_Idro > Incentivo_Totale_Condivisione): 
    print('ATTENZIONE LA QUOTA PER L\'IDROELETTRICO SUPERA L\'INCENTIVO TOTALE ALLA CONDIVISIONE')
  Incentivo_Attribuibile = Incentivo_Attribuibile - Quota_Idro
else: 
  Quota_Idro = 0

######## ATTENZIONE: AD OGNI STEP LA SOMMA DEI DUE NUMERI DEVE ESSERE 1
Incentivo_Attribuibile = 0.90*(Incentivo_Totale_Condivisione - Quota_Idro)
Incentivo_Costi_Gestione = 0.10*Incentivo_Totale_Condivisione
#########
Quota_Fissa = (0.20*Incentivo_Attribuibile) #ATTENZIONE: QUESTA QUOTA E' DA DIVIDERE IN PARTI UGUALI FRA I MEMBRI DELLA COMUNITA'.
Quota_Variabile = 0.80*Incentivo_Attribuibile
#########
Quota_Consumatori = 0.60*Quota_Variabile 
Quota_Immissione = 0.40*Quota_Variabile
#########
Quota_Produttore = 0.98*Quota_Immissione
Quota_Prosumer = 0.02*Quota_Immissione
######### GESTISCO ECCEZIONE IN CUI NON CI SONO PROSUMER PER NON PERDERE QUOTE DI INCENTIVO
if(numero_prosumer == 0): 
  Quota_Produttore = 1*Quota_Immissione
  Quota_Prosumer = 0*Quota_Immissione

In [ ]:
print('Incentivo Totale Condivisione: '+str(Incentivo_Totale_Condivisione))
print(f'Quota Idro: {Quota_Idro}')
print(f'Incentivo Attribuibile: {Incentivo_Attribuibile}')
print(f'Incentivo Costi di Gestione: {Incentivo_Costi_Gestione}')
print(f'Quota Fissa: {Quota_Fissa}')
print(f'Quota Variabile: {Quota_Variabile}')
print(f'Quota Produttore: {Quota_Produttore}')
print(f'Quota Consumatori: {Quota_Consumatori}')

Incentivo Totale Condivisione: 16025.182600000044
Quota Idro: 289.03000000000003
Incentivo Attribuibile: 14162.537340000039
Incentivo Costi di Gestione: 1602.5182600000044
Quota Fissa: 2832.507468000008
Quota Variabile: 11330.029872000032
Quota Produttore: 4441.371709824012
Quota Consumatori: 6798.017923200019


In [ ]:
if Incentivo_Totale_Condivisione == Incentivo_Attribuibile + Incentivo_Costi_Gestione + Quota_Idro: 
  print('C\'è congruenza')

In [ ]:
print('Quota Fissa ' +str(Quota_Fissa))
print('Quota Immissione ' +str(Quota_Immissione))
print('Quota Consumatori ' +str(Quota_Consumatori))
print('Quota Produttore ' +str(Quota_Produttore))
print('Quota Prosumer ' +str(Quota_Prosumer))

Quota Fissa 2832.507468000008
Quota Immissione 4532.011948800013
Quota Consumatori 6798.017923200019
Quota Produttore 4441.371709824012
Quota Prosumer 90.64023897600026


In [ ]:
def CalcolaIncentivo(copy): 
    #copy = df.copy()
    if(copy['Categoria'] == 'Consumer'): 
      #print('Nome: '+str(copy['Identificativo']) + ' Categoria: ' +str(copy['Categoria']))
      #print('quota pro capite: '+ str(QuotaProCapite))
      #print('percentuale condivisa: '+ str(Quota_Produttore*copy['Percentuale Condivisa']))
      #print('Totale suo incentivo: '+str( QuotaProCapite + Quota_Consumatori*copy['Percentuale Condivisa']))
      #print('##########################################################################################')
      return QuotaProCapite + Quota_Consumatori*copy['Percentuale Condivisa']


    elif(copy['Categoria'] == 'Prosumer'): 
      #print('Nome: '+str(copy['Identificativo']) + ' Categoria: ' +str(copy['Categoria']))
      #print('quota pro capite: '+ str(QuotaProCapite))
      #print('percentuale immessa: '+ str(Quota_Produttore*copy['Percentuale Immessa']))
      #print('percentuale immessa prosumer: '+str(Quota_Prosumer*copy['Percentuale Immessa Prosumer']))
      #print('Totale suo incentivo: '+str( QuotaProCapite + Quota_Produttore*copy['Percentuale Immessa'] + Quota_Prosumer*copy['Percentuale Immessa Prosumer']))
      #print('##########################################################################################')
      return QuotaProCapite + Quota_Consumatori*copy['Percentuale Condivisa'] + Quota_Produttore*copy['Percentuale Immessa'] + Quota_Prosumer*copy['Percentuale Immessa Prosumer']


    elif(copy['Categoria'] == 'Produttore'): 
      #print('Nome: '+str(copy['Identificativo']) + ' Categoria: ' +str(copy['Categoria']))
      #print('quota pro capite: '+ str(QuotaProCapite))
      #print('percentuale immessa: '+ str(Quota_Produttore*copy['Percentuale Immessa']))
      #print('Totale suo incentivo: '+str( QuotaProCapite + Quota_Produttore*copy['Percentuale Immessa']))
      #print('##########################################################################################')                ''' INSERIMENTO IDROELETTRICO'''
      if copy['Identificativo'] == 'Idroelettrico': 
        return Quota_Idro ########################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!########## Definire a cosa attinge il produttore idroelettrico. 
      return  QuotaProCapite + Quota_Produttore*copy['Percentuale Immessa']
    else: 
      print('non rientra in nessuna categoria ERRORE')

In [ ]:
Quota_Fissa

2832.507468000008

In [ ]:
numero_partecipanti

3

In [ ]:
#Consumatori: Quota fissa + % Quota Consumatori
#Produttore: Quota fissa + % Fissa su Quota Variabile
#df_finale['Incentivo Condivisione'] = df_finale.apply(lambda x: (Quota_Fissa/numero_partecipanti + x['Percentuale Condivisa']*Quota_Consumatori) if (x['Categoria'] == 'Consumer')  else ( (Quota_Fissa/numero_partecipanti + Quota_Produttore/(numero_produttori+ numero_prosumer) if (x['Categoria'] == 'Produttore') else (Quota_Fissa/numero_partecipanti + Quota_Prosumer/numero_prosumer + Quota_Produttore/(numero_produttori+numero_prosumer), axis =1)

#df_finale['Incentivo Condivisione'] = df_finale.apply(lambda x: Quota_Fissa/numero_partecipanti + ( (Quota_Consumatori*x['Percentuale Condivisa']) if (x['Categoria'] == 'Consumer')  else ( (Quota_Produttore*x['Percentuale Immessa'] + Quota_Prosumer*x['Percentuale Immessa Prosumer']) if(x['Categoria'] == 'Prosumer') else (Quota_Produttore*x['Percentuale Immessa']))), axis =1)
QuotaProCapite = Quota_Fissa/numero_partecipanti #La quota pro capite viene attribuita nella funzione Calcola incentivo. 

df_finale['Incentivo Condivisione'] = df_finale.apply(CalcolaIncentivo, axis = 1)

#df_finale['Incentivo Condivisione '] = df_finale.apply(lambda x: QuotaProCapite + Quota_Produttore*x['Percentuale Immessa'] if (x['Categoria'] == 'Produttore')  else (  (QuotaProCapite + Quota_Produttore*x['Percentuale Immessa'] + Quota_Prosumer*x['Percentuale Immessa Prosumer']) if(x['Categoria'] == 'Prosumer') else (QuotaProCapite + Quota_Consumatori*x['Percentuale Condivisa'])), axis=1)

df_finale['Ritiro Dedicato'] = df_finale.apply(lambda x: (x['Somma Immissione']*ritiro_dedicato) if (x["Somma Immissione"] > 0 and x['Identificativo'] != 'Idroelettrico')  else 0, axis =1) #N.B viene escluso il produttore idroelettrico

df_finale['Vendita Energia Idroelettrico'] = df_finale.apply(lambda x: (x['Somma Immissione']*val_prod_idro) if (x['Identificativo'] == 'Idroelettrico')  else 0, axis =1) #Calcolo vedita alla rete energia idroelettrico secondo parametro settato "val_prod_idro"

Ritiro_Dedicato_Totale = df_finale['Ritiro Dedicato'].sum()

df_finale['Incentivo Totale'] = df_finale['Incentivo Condivisione'] + df_finale['Ritiro Dedicato'] + df_finale['Vendita Energia Idroelettrico'] #Calcolo incentivo totale singolo utente. N.B. quota pro capite attribuita in funzione calcola incentivo 
df_finale

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,...,Costo Bolletta,Percentuale Condivisa,Taglia Impianto,Categoria,Percentuale Immessa,Percentuale Immessa Prosumer,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Incentivo Totale
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,...,1.041e+05,0.04533,NaN,Prosumer,0.001389,0.001389,1.259e+03,371.6,0.0,1.63e+03
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,...,8.007e+03,0.0,NaN,Prosumer,0.9986,0.9986,5.47e+03,2.671e+05,0.0,2.726e+05
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,...,1.041e+05,0.0,NaN,Produttore,0.0,0.0,289.0,0.0,371.6,660.6
3,Produttore,NaN,0.0,NaN,-2.836e+06,NaN,NaN,0.0,-2.836e+06,-2.269e+03,...,0.0,0.9547,-2.269e+03,Produttore,0.0,0.0,944.2,0.0,0.0,944.2


***Conto Economico della CE***

Voci di Costo

#5) Parametri di Ammortamento

Elenco voci di costo e tempi di ammortamento

In [ ]:
Architettura_CER = 400 #€/partecipante una tantum
Tempo_Ammortamento_Architettura = 4 #Anni
####################################################################
Commissione = 48 #€/partecipante all'anno
####################################################################
Ammortamento_Impianto_Fotovoltaico = 0.00 # % del costo dell'impianto
Tempo_Ammortamento_Fotovoltaico = 10 #Anni


In [ ]:
d = {'Architettura': [Architettura_CER], 'Tempo_Ammortamento_Architettura': [Tempo_Ammortamento_Architettura], 'Tempo_Ammortamento_Architettura': [Tempo_Ammortamento_Architettura], 'Commissione': [Commissione], 'Ammortamento_Impianto_Fotovoltaico': [Ammortamento_Impianto_Fotovoltaico], 'Tempo_Ammortamento_Fotovoltaico': [Tempo_Ammortamento_Fotovoltaico]}
Parametri_Ammortamento = pd.DataFrame(data = d)

In [ ]:
Parametri_Ammortamento

,Architettura,Tempo_Ammortamento_Architettura,Commissione,Ammortamento_Impianto_Fotovoltaico,Tempo_Ammortamento_Fotovoltaico
0,400,4,48,0.0,10


Flussi di cassa CER

In [ ]:
Flussi_Di_Cassa_CER = (Incentivo_Totale_Condivisione + Ritiro_Dedicato_Totale)
Flussi_Di_Cassa_CER

283523.59030000004

Trattenute condivisione

In [ ]:
Incentivo_Costi_Gestione

1602.5182600000044

Costo totale per la costruzione dei nuovi impianti della CER

In [ ]:
df_finale

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,...,Costo Bolletta,Percentuale Condivisa,Taglia Impianto,Categoria,Percentuale Immessa,Percentuale Immessa Prosumer,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Incentivo Totale
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,...,1.041e+05,0.04533,NaN,Prosumer,0.001389,0.001389,1.259e+03,371.6,0.0,1.63e+03
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,...,8.007e+03,0.0,NaN,Prosumer,0.9986,0.9986,5.47e+03,2.671e+05,0.0,2.726e+05
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,...,1.041e+05,0.0,NaN,Produttore,0.0,0.0,289.0,0.0,371.6,660.6
3,Produttore,NaN,0.0,NaN,-2.836e+06,NaN,NaN,0.0,-2.836e+06,-2.269e+03,...,0.0,0.9547,-2.269e+03,Produttore,0.0,0.0,944.2,0.0,0.0,944.2


In [ ]:
Costo_Totale_Impianti = df_finale[df_finale['Categoria']=='Produttore']['Costo Impianto Potenziale'].sum()
Costo_Totale_Impianti

-2722368.675199999

Costi di Funzionamento = commissione + costo architettura + rimborso quota impianti a produttori

In [ ]:
Costi_Di_Funzionamento = (Commissione * numero_partecipanti) + ((Architettura_CER * numero_partecipanti)/Tempo_Ammortamento_Architettura) #Sarebbero da considerare separati
Costi_Di_Impianto = (Costo_Totale_Impianti*Ammortamento_Impianto_Fotovoltaico/Tempo_Ammortamento_Fotovoltaico)


Costi_Di_Funzionamento_Produttore = ( (Costi_Di_Funzionamento)/numero_partecipanti ) * numero_produttori
Costi_Di_Funzionamento_Consumer = ((Costi_Di_Funzionamento-Costi_Di_Funzionamento_Produttore))

In [ ]:
print('Costi funzionamento + Impianto: '+str(Costi_Di_Funzionamento+Costi_Di_Impianto))
print('Costi di funzionamento consumer: '+str(Costi_Di_Funzionamento_Consumer))
print('Costi di funzionamento produttore: '+str(Costi_Di_Funzionamento_Produttore))


Costi funzionamento + Impianto: 444.0
Costi di funzionamento consumer: 148.0
Costi di funzionamento produttore: 296.0


In [ ]:
Trattenute_Incentivi = Incentivo_Costi_Gestione
Trattenute_Incentivi

1602.5182600000044

Bilancio Economico CER

In [ ]:
Margine_CER = Incentivo_Attribuibile - Costi_Di_Funzionamento - Costi_Di_Impianto
Margine_CER

13718.537340000039

In [ ]:
Bilancio_Economico_CER = Flussi_Di_Cassa_CER - Costi_Di_Funzionamento - Costi_Di_Impianto - Trattenute_Incentivi
Bilancio_Economico_CER

281477.07204000006

In [ ]:
df_finale[df_finale['Identificativo']!='Produttore']['Incentivo Condivisione'].sum()

7017.56684532546

Calcolo tempo di rientro degli investimenti impianti

In [ ]:
# Entrata annuale per il produttore fotovoltaico derivante dall'ammortamento della percentuale dell'impianto sui componenti della CE
df_finale['Rata CER Impianto'] = df_finale.apply(lambda x: ( (x['Costo Impianto Potenziale']*(Ammortamento_Impianto_Fotovoltaico)/Tempo_Ammortamento_Fotovoltaico) ) if ((x["Categoria"] == 'Produttore') and x['Identificativo'] != 'Idroelettrico')  else (0) , axis =1)

# Ripartisco i costi di funzionamento della CE: Produttore paga i costi di funzionamento produttore, I consumer pagano i costi di funzionamento consumer, Idroelettrico non paga nulla
df_finale['Costi Funzionamento CER'] = df_finale.apply(lambda x: (0) if (x['Identificativo'] == 'Idroelettrico') else ( (Costi_Di_Funzionamento_Produttore/numero_produttori) if (x['Categoria'] == 'Produttore') else ( Costi_Di_Funzionamento_Consumer *( x['Incentivo Condivisione'] / df_finale[df_finale['Identificativo']!='Produttore']['Incentivo Condivisione'].sum())) ), axis = 1)

# Ripartisco i costi di Impianto fra i membri della CE (0 per produttore perché non può pagarsi il suo impianto e proporzionale all'incentivo di condivisione per gli altri membri)
df_finale['Contributo Alla Produzione'] = df_finale.apply(lambda x: ( ( 0 ) ) if ((x["Categoria"] == 'Produttore'))  else ( Costi_Di_Impianto *( x['Incentivo Condivisione'] / (df_finale[df_finale['Identificativo']!='Produttore']['Incentivo Condivisione'].sum()) )) , axis =1)
# 
df_finale['Rientro Investimento Impianto'] = df_finale.apply(lambda x: ( (x['Costo Impianto Potenziale']) / (x['Incentivo Condivisione'] + x['Ritiro Dedicato'] + x['Rata CER Impianto'] - x['Costi Funzionamento CER']) ) if ((x["Categoria"] == 'Produttore') & (x['Identificativo'] != 'Idroelettrico'))  else (0) , axis =1)

df_finale['Beneficio Economico Totale'] = df_finale.apply(lambda x: ( x['Incentivo Totale'] + x['Rata CER Impianto'] - x['Costi Funzionamento CER']) if (x["Categoria"] == 'Produttore')  else (x['Vendita Energia Idroelettrico'] + x['Incentivo Condivisione'] if x['Identificativo'] == 'Idroelettrico' else (x['Incentivo Totale'] - x['Costi Funzionamento CER'] - x['Contributo Alla Produzione'])) , axis =1)

# Calcolo proporzione pagamento incentivo condivisione per calcolare quanto uno paga il gestore della cer (% sulle trattenute incentivo condivisione)
inc_cond_no_prod = df_finale[df_finale['Categoria'] != 'Produttore']['Incentivo Condivisione'].sum() #totale incentivo condivisione, produttore escluso
df_finale['Percentuale Trattenuta Incentivo Condivisione'] = df_finale.apply(lambda x: ( ( x['Incentivo Condivisione'] / inc_cond_no_prod ) if (x['Categoria'] != 'Produttore') else (0) ), axis =1) #percentuale incentivo condivisione del singolo utente sul totale 
df_finale['Trattenuta Incentivo Condivisione'] = df_finale.apply(lambda x: ( (  Trattenute_Incentivi *(x['Percentuale Trattenuta Incentivo Condivisione'] )) ), axis =1) #Calcolo quanto un utente deve fornire al produttore per il contributo alla produzione proporzionalmente all'incentivo condivisione percepito

df_finale.drop(columns = ['Rata CER Impianto'])

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,...,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Incentivo Totale,Costi Funzionamento CER,Contributo Alla Produzione,Rientro Investimento Impianto,Beneficio Economico Totale,Percentuale Trattenuta Incentivo Condivisione,Trattenuta Incentivo Condivisione
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,...,1.259e+03,371.6,0.0,1.63e+03,26.54,-0.0,0.0,1.604e+03,0.1871,299.8
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,...,5.47e+03,2.671e+05,0.0,2.726e+05,115.4,-0.0,0.0,2.725e+05,0.8129,1.303e+03
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,...,289.0,0.0,371.6,660.6,0.0,0.0,0.0,660.6,0.0,0.0
3,Produttore,NaN,0.0,NaN,-2.836e+06,NaN,NaN,0.0,-2.836e+06,-2.269e+03,...,944.2,0.0,0.0,944.2,148.0,0.0,-3.419e+03,796.2,0.0,0.0


In [ ]:
df_finale

,Identificativo,Potenza Da Installare,Somma Fascia 1,Somma Fascia 2,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Consumo Per Fotovoltaico,Differenza,Taglia Impianto Potenziale,...,Ritiro Dedicato,Vendita Energia Idroelettrico,Incentivo Totale,Rata CER Impianto,Costi Funzionamento CER,Contributo Alla Produzione,Rientro Investimento Impianto,Beneficio Economico Totale,Percentuale Trattenuta Incentivo Condivisione,Trattenuta Incentivo Condivisione
0,A,16.0,1.157e+05,6.27e+04,4.129e+03,0.0,2.507e+04,1.388e+05,-1.347e+05,107.7,...,371.6,0.0,1.63e+03,0.0,26.54,-0.0,0.0,1.604e+03,0.1871,299.8
1,B,2e+03,9.322e+04,9.829e+04,2.968e+06,1.129e+05,3.081e+06,-1.014e+03,2.969e+06,-2.375e+03,...,2.671e+05,0.0,2.726e+05,0.0,115.4,-0.0,0.0,2.725e+05,0.8129,1.303e+03
2,Idroelettrico,0.0,0.0,0.0,4.129e+03,0.0,0.0,0.0,1.376e+03,-1.101,...,0.0,371.6,660.6,0.0,0.0,0.0,0.0,660.6,0.0,0.0
3,Produttore,NaN,0.0,NaN,-2.836e+06,NaN,NaN,0.0,-2.836e+06,-2.269e+03,...,0.0,0.0,944.2,-0.0,148.0,0.0,-3.419e+03,796.2,0.0,0.0


Rimanipolazione finale organizzazione DF

In [ ]:
#Cambio di segno la colonna contributo alla produzione
df_finale = df_finale.fillna(0)
df_finale['Contributo Alla Produzione']= -df_finale['Contributo Alla Produzione']
#Inserisco il valore di rata cer impianto di produttore nella colonna contributo alla produzione
df_finale.loc[df_finale['Categoria'] =='Produttore', 'Contributo Alla Produzione'] = df_finale[df_finale['Categoria'] =='Produttore']['Rata CER Impianto']
#Drop della colonne non volute
if(numero_produttori > 0): 
  df_finale.drop(['Rata CER Impianto', 'Somma Fascia 1', 'Somma Fascia 2', 'Taglia Impianto', 'Consumo Per Fotovoltaico'], axis=1, inplace=True)
else:
  df_finale.drop(['Rata CER Impianto', 'Somma Fascia 1', 'Somma Fascia 2', 'Consumo Per Fotovoltaico'], axis=1, inplace=True)

In [ ]:
#Rinomino "Differenza" in "Bilancio Energetico", Incentivo Totale in Entrate Totali e Trattentuta Incentivo Condivisione in Costi Gestione CER
df_finale = df_finale.rename(columns = {'Differenza': 'Bilancio Energetico', 'Incentivo Totale': 'Entrate Totali', 'Trattenuta Incentivo Condivizione': 'Costi Gestione CER'})

#6) Tabella riassuntiva finale

In [ ]:
#df_finale.style.format(thousands = '.', precision = 0)
df_finale.style.format(thousands = ',', precision = 2)

,Identificativo,Potenza Da Installare,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Bilancio Energetico,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,Percentuale Condivisa,Categoria,Percentuale Immessa,Percentuale Immessa Prosumer,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Entrate Totali,Costi Funzionamento CER,Contributo Alla Produzione,Rientro Investimento Impianto,Beneficio Economico Totale,Percentuale Trattenuta Incentivo Condivisione,Trattenuta Incentivo Condivisione
0,A,16.00,"4,129.00",0.00,"25,074.14","-134,665.40",107.73,"129,278.78","19,200.00","104,145.52",0.05,Prosumer,0.00,0.00,"1,258.65",371.61,0.00,"1,630.26",26.54,0.00,0.00,"1,603.72",0.19,299.77
1,B,"2,000.00","2,968,075.53","112,874.16","3,080,949.69","2,969,089.77","-2,375.27",0.00,"2,400,000.00","8,006.68",0.00,Prosumer,1.00,1.00,"5,469.89","267,126.80",0.00,"272,596.68",115.36,0.00,0.00,"272,481.32",0.81,"1,302.75"
2,Idroelettrico,0.00,"4,129.00",0.00,0.00,"1,376.33",-1.10,0.00,0.00,"104,145.52",0.00,Produttore,0.00,0.00,289.03,0.00,371.61,660.64,0.00,0.00,0.00,660.64,0.00,0.00
3,Produttore,0.00,"-2,835,800.70",0.00,0.00,"-2,835,800.70","-2,268.64","-2,722,368.68",0.00,0.00,0.95,Produttore,0.00,0.00,944.17,0.00,0.00,944.17,148.00,-0.00,"-3,419.33",796.17,0.00,0.00


Confronto Scambio sul Posto

In [ ]:
df_ssp.style.format(thousands = ',', precision = 2)

,Identificativo,Totale Produzione,Totale Autoconsumo,Totale Esportata,Totale Importata,Beneficio SSP
0,Ciemme srl,"18,969.53","13,133.60","7,180.00","19,009.00","1,077.00"
1,Latteria del Molise Srl,"49,894.31","37,151.20","4,129.04","46,700.08",619.36


In [ ]:
df_finale

,Identificativo,Potenza Da Installare,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Bilancio Energetico,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,...,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Entrate Totali,Costi Funzionamento CER,Contributo Alla Produzione,Rientro Investimento Impianto,Beneficio Economico Totale,Percentuale Trattenuta Incentivo Condivisione,Trattenuta Incentivo Condivisione
0,A,16.0,4.129e+03,0.0,2.507e+04,-1.347e+05,107.7,1.293e+05,1.92e+04,1.041e+05,...,1.259e+03,371.6,0.0,1.63e+03,26.54,0.0,0.0,1.604e+03,0.1871,299.8
1,B,2e+03,2.968e+06,1.129e+05,3.081e+06,2.969e+06,-2.375e+03,0.0,2.4e+06,8.007e+03,...,5.47e+03,2.671e+05,0.0,2.726e+05,115.4,0.0,0.0,2.725e+05,0.8129,1.303e+03
2,Idroelettrico,0.0,4.129e+03,0.0,0.0,1.376e+03,-1.101,0.0,0.0,1.041e+05,...,289.0,0.0,371.6,660.6,0.0,0.0,0.0,660.6,0.0,0.0
3,Produttore,0.0,-2.836e+06,0.0,0.0,-2.836e+06,-2.269e+03,-2.722e+06,0.0,0.0,...,944.2,0.0,0.0,944.2,148.0,-0.0,-3.419e+03,796.2,0.0,0.0


***Cash Flow Produttori***

In [ ]:
#SPECIFICARE DISCOUNT RATE PER CALCOLO IRR, NPV, IP E ATTUALIZZAZIONE FLUSSI DI CASSA. 
discount_rate = 0.03

if numero_produttori > 0:
  #QUESTO LO USO PER IL CASHFLOW CUMULATO
  df_discounted_cashflow = df_finale.loc[(df_finale['Categoria'] == 'Produttore') & (df_finale['Identificativo'] != 'Idroelettrico'), ['Identificativo', 'Costo Impianto Potenziale', 'Beneficio Economico Totale']]
  df_discounted_cashflow = df_discounted_cashflow.rename(columns = {'Costo Impianto Potenziale': 'Cumulato', 'Beneficio Economico Totale': 'Entrata'})
  df_discounted_cashflow['Anno'] = 0
  df_discounted_cashflow['Cumulato'] = - df_discounted_cashflow['Cumulato']
  #df_cashflow = df_cashflow.drop(columns = ['Taglia Impianto', 'Somma Fascia 1', 'Consumo Per Fotovoltaico', 'Produzione', Di])

  #QUESTO LO USO PER IL CASHFLOW (NON CUMULATO)
  df_cashflow = df_finale.loc[(df_finale['Categoria'] == 'Produttore') & (df_finale['Identificativo'] != 'Idroelettrico'), ['Identificativo', 'Costo Impianto Potenziale', 'Beneficio Economico Totale']]
  df_cashflow = df_cashflow.rename(columns = {'Costo Impianto Potenziale': 'Cashflow', 'Beneficio Economico Totale': 'Entrata'})
  df_cashflow['Anno'] = 0
  df_cashflow['Cashflow'] = - df_cashflow['Cashflow']

  def CalcolaCashFlow(df, anni): #Funzione per il calcolo del cashflow (immagazzinato nel df_cashflow_finale)
    df = pd.DataFrame(np.repeat(df.values, anni, axis = 0), columns = df.columns)
    for index, row in df.iterrows(): 
      #print(index)
      if(index == 0): #Salto l'anno 0 -> ho solo spese: investimento iniziale
        #df.iloc[index]['Cashflow'] += df.iloc[index]['Entrata']
        continue
      df.iloc[index]['Cashflow'] = df.iloc[index]['Entrata']
      row['Anno'] = index
      #print(df)
    return df

  def CalcolaDiscountedCashFlow(df, anni, r):  #Funzione per il calcolo del cashflow cumulato (df_discounted_cashflow_finale)
    entrata_base = df.iloc[0]['Entrata']
    df = pd.DataFrame(np.repeat(df.values, anni, axis = 0), columns = df.columns)
    for index, row in df.iterrows(): 
      #print(index)
      if(index == 0): #Salto l'anno 0 -> ho solo spese: investimento iniziale
        continue
      df.iloc[index]['Entrata'] = entrata_base/pow((1+r), int(index))
      df.iloc[index]['Cumulato'] = df.iloc[index-1]['Cumulato'] + df.iloc[index]['Entrata']
      row['Anno'] = index
      #print(df)
    return df

  #CALCOLO CASHFLOW
  #CHIAMO FUNZIONE: ARGOMENTI: X (DA NON CAMBIARE), NUMERO: ANNI PER CUI SI VUOLE CALCOLARE IL CASHFLOW
  df_cashflow_finale = pd.DataFrame()
  df_cashflow_finale = df_cashflow_finale.append(df_cashflow.groupby('Identificativo').apply(lambda x: CalcolaCashFlow(x, 20)))
  #RISTABILISCO UN DF ORDINATO
  df_cashflow_finale = df_cashflow_finale.drop(columns = ['Identificativo']).reset_index()
  df_cashflow_finale = df_cashflow_finale.drop(columns = ['level_1'])
  print(df_cashflow_finale)

  #CALCOLO DISCOUNTED CASHFLOW
  #CHIAMO FUNZIONE: ARGOMENTI: X (DA NON CAMBIARE), NUMERO: ANNI PER CUI SI VUOLE CALCOLARE IL CASHFLOW
  df_discounted_cashflow_finale = pd.DataFrame()
  df_discounted_cashflow_finale = df_discounted_cashflow_finale.append(df_discounted_cashflow.groupby('Identificativo').apply(lambda x: CalcolaDiscountedCashFlow(x, 20, discount_rate)))
  #RISTABILISCO UN DF ORDINATO
  df_discounted_cashflow_finale = df_discounted_cashflow_finale.drop(columns = ['Identificativo']).reset_index()
  df_discounted_cashflow_finale = df_discounted_cashflow_finale.drop(columns = ['level_1'])
  print(df_discounted_cashflow_finale)

   Identificativo  Cashflow Entrata Anno
0      Produttore 2.722e+06   796.2    0
1      Produttore     796.2   796.2    1
2      Produttore     796.2   796.2    2
3      Produttore     796.2   796.2    3
4      Produttore     796.2   796.2    4
5      Produttore     796.2   796.2    5
6      Produttore     796.2   796.2    6
7      Produttore     796.2   796.2    7
8      Produttore     796.2   796.2    8
9      Produttore     796.2   796.2    9
10     Produttore     796.2   796.2   10
11     Produttore     796.2   796.2   11
12     Produttore     796.2   796.2   12
13     Produttore     796.2   796.2   13
14     Produttore     796.2   796.2   14
15     Produttore     796.2   796.2   15
16     Produttore     796.2   796.2   16
17     Produttore     796.2   796.2   17
18     Produttore     796.2   796.2   18
19     Produttore     796.2   796.2   19
   Identificativo  Cumulato Entrata Anno
0      Produttore 2.722e+06   796.2    0
1      Produttore 2.723e+06   773.0    1
2      Produttor

In [ ]:
#FUNZIONE PER IL CALCOLO DEL NPV
def net_pv(cfList, r):
    sum_pv = 0  # <-- variable used to sum result
    for i, pmt in enumerate(cfList, start=1):  # <-- use of enumerate allows you to do away with the counter variables.
        sum_pv += pmt / ((1 + r) ** i)  # <-- add pv of one of the cash flows to the sum variable
    return sum_pv  # <-- only return the sum after your loop has completed.

In [ ]:
def profitability_index(npv, initial_investment): 
    return (npv+initial_investment)/initial_investment

In [ ]:
#ATTENZIONE LA FUNZIONE NPV INTEGRATA IN NUMPY NON TORNA UN VALORE CONGRUO CON QUELLO CALCOLATO IN EXCEL. 
n = df_cashflow_finale.to_numpy() 
a = npf.npv(discount_rate, n[:, 1])
a

2733772.8422451513

#7) KPI Economici

In [ ]:
npv = net_pv(n[:,1], discount_rate) #npf.npv(discount_rate, n[:,1])# il npv calcolato con la numpy financial library torna un valore differente da quello di excel e della funzione realizzata
irr = npf.irr(n[:,1])
#pi = profitability_index(npv, abs(n[0,1]))
print('npv: '+str(npv)+', irr: '+str(irr))#+', pi: '+str(pi))

npv: 2654148.3905292726, irr: nan


# 8) Scrittura file output

In [ ]:
#Scrittura in output
try: 
    file = Path(directory+'/output_DER.xlsx')
    if file.exists():
       print("Il file esiste già. Elimino quello vecchio.") 
       os.remove(file)   
    with pd.ExcelWriter(directory+'/output_DER.xlsx') as writer:  
        df_finale.to_excel(writer, sheet_name='Tabella riassuntiva finale', index = False)
        df_cashflow_finale.to_excel(writer, sheet_name='Cashflow produttore', index = False)
        df_discounted_cashflow_finale.to_excel(writer, sheet_name='Discounted cashflow produttore', index = False)
        print('Scrittura del file di output realizzata con successo')
except: 
    raise TypeError('Impossibile scrivere il file, verificare che non sia già aperto e riprovare.')

Il file esiste già. Elimino quello vecchio.
Scrittura del file di output realizzata con successo


# Grafici

In [ ]:
#Import pacchetti grafici
#Pacchetti grafica
!pip install --upgrade plotly
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
!pip install chart_studio
import chart_studio.plotly as py
from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 6.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 632 kB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=5abdce440e2662b18fa5269444d4c80064e248866c41a9956b058246a3c89dd2
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
if numero_produttori >0: 
  fig = px.line(df_discounted_cashflow_finale, x='Anno', y='Cumulato', color='Identificativo', markers=True, text = "Cumulato")
  fig.update_layout(
      title='Cashflow Cumulativo Con Flussi Di Cassa Attualizzati',
      xaxis_tickfont_size=14,
      xaxis = dict(
          showgrid =True,
          zeroline = True, 
          nticks = 25, 
          showline = True,
          title='Anni',
          titlefont_size=16,
          tickfont_size=14,
      ),
      yaxis=dict(
          showgrid =True,
          zeroline = True, 
          nticks = 30, 
          showline = True,
          title='€',
          titlefont_size=16,
          tickfont_size=14,
      ),  
  ) 
  fig.update_traces(textposition="bottom right")
  fig.update_yaxes(zeroline=True, zerolinewidth=1.5, zerolinecolor='grey')
  fig.update_traces(texttemplate='%{text:.2f}€')
  fig.update_layout(xaxis_range=[-0.5,df_discounted_cashflow_finale.Anno.max()+1.5])
  #fig.update_traces(texttemplate= format('%{value:,.0f}€'))
  fig.show()

In [ ]:
df_finale[df_finale['Categoria'] == 'Idroelettrico']

,Identificativo,Potenza Da Installare,Somma Immissione,Somma Autoconsumo,Somma Produzione Aggiuntiva,Bilancio Energetico,Taglia Impianto Potenziale,Costo Impianto Potenziale,Costo Potenza da Installare,Costo Bolletta,...,Incentivo Condivisione,Ritiro Dedicato,Vendita Energia Idroelettrico,Entrate Totali,Costi Funzionamento CER,Contributo Alla Produzione,Rientro Investimento Impianto,Beneficio Economico Totale,Percentuale Trattenuta Incentivo Condivisione,Trattenuta Incentivo Condivisione


In [ ]:
fig = make_subplots(
    rows=6, cols=3,
    specs=[[{"rowspan": 1, "colspan": 3},None, None],
          [{"rowspan": 1, "colspan": 3},None, None], 
          [{"rowspan": 1, "colspan": 1}, {"rowspan": 1, "colspan": 2},None], 
          [{"rowspan": 1, "colspan": 1}, {"rowspan": 1, "colspan": 2},None],
          [{"rowspan": 1, "colspan": 1}, {"rowspan": 1, "colspan": 2},None], 
          [{"rowspan": 1, "colspan": 1}, {"rowspan": 1, "colspan": 2},None]], 
          shared_yaxes=True)

fig.add_trace(go.Bar(x=['Totale Ritiro Dedicato'], y=[df_finale['Ritiro Dedicato'].sum()],
                    marker_color=['blue'], text=[df_finale['Ritiro Dedicato'].sum().round(0)]),
              1, 1)


fig.add_trace(go.Bar(x=['Quota Idroelettrico'], y=[df_finale[df_finale['Identificativo'] == 'Idroelettrico']['Entrate Totali'].sum()],
                    marker_color=['pink'], text=[df_finale[df_finale['Identificativo'] == 'Idroelettrico']['Entrate Totali'].sum().round(0)]),
              2, 1)


fig.add_trace(go.Bar(x=['Totale Incentivo Energia Condivisa'], y=[Incentivo_Totale_Condivisione],
                   marker_color=['#00FF6F'], text=[Incentivo_Totale_Condivisione.round(0)]),
              3, 1)
fig.add_trace(go.Bar(x=['Costi di Gestione', 'Incentivo Attribuibile'], y=[Incentivo_Costi_Gestione, Incentivo_Attribuibile],
                    marker_color=['orange', 'red'], text=[Incentivo_Costi_Gestione.round(0), Incentivo_Attribuibile.round(0)]),
              3, 2)

fig.add_trace(go.Bar(x=['Incentivo Attribuibile'], y=[Incentivo_Attribuibile],
                    marker_color=['red'], text=[Incentivo_Attribuibile.round(0)]),
              4, 1)
fig.add_trace(go.Bar(x=['Quota Fissa', 'Quota Variabile'], y=[Quota_Fissa, Quota_Variabile],
                    marker_color=['yellow', 'purple'], text=[Quota_Fissa.round(0), Quota_Variabile.round(0)]),
              4, 2)

fig.add_trace(go.Bar(x=['Quota Variabile'], y=[Quota_Variabile],
                    marker_color=['purple'], text=[Quota_Variabile.round(0)]),
              5, 1)

fig.add_trace(go.Bar(x=['Quota Consumatori', 'Quota Immissione'], y=[Quota_Consumatori, Quota_Immissione],
                    marker_color=['pink', '#4E8FFF'], text=[Quota_Consumatori.round(0), Quota_Immissione.round(0)]),
              5, 2)

fig.add_trace(go.Bar(x=['Quota Immissione'], y=[Quota_Immissione],
                    marker_color=['#4E8FFF'], text=[Quota_Immissione.round(0)]),
              6, 1)

fig.add_trace(go.Bar(x=['Quota Produttore', 'Quota Prosumer'], y=[Quota_Produttore, Quota_Prosumer],
                    marker_color=['#FF5733', '#D58BB9'], text=[Quota_Produttore.round(0), Quota_Prosumer.round(0)]),
              6, 2)

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.update_layout(barmode='stack')
fig.update_layout(height=700, width=900)
fig.update_traces(texttemplate= format('%{value:,.0f}€'))
fig.show()

In [ ]:

parziale = df_finale[df_finale['Identificativo'] != 'Produttore']
fig = go.Figure()
fig.update_layout(
    title=' ',
    xaxis_tickfont_size=14,
    xaxis = dict(
        showgrid =True,
        zeroline = True, 
        nticks = 20, 
        showline = True,
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        showgrid =True,
        zeroline = True, 
        nticks = 20, 
        showline = True,
        title='kWh',
        titlefont_size=16,
        tickfont_size=14,
    ),
    barmode='group',
    bargap=0.2, # gap between bars of adjacent location coordinates.
    bargroupgap=0 # gap between bars of the same location coordinate.
)
#Risparmio da Autoconsumo Istantaneo
#fig.add_trace(go.Bar(x=parziale['Identificativo'], y=parziale['Produzione'],
#                base=0,
#                marker_color='red',
#                name='Produzione'
#                ))
#Incentivo Condivisione Arera
#fig.add_trace(go.Bar(x=parziale['Identificativo'], y=parziale['Consumo Per Fotovoltaico'],
#                base=0,
#                marker_color='#ff8000',
#                name='Consumo Per Fotovoltaico', 
#                ))
#Incentivo Condivisione Mise
fig.add_trace(go.Bar(x=parziale['Identificativo'], y=parziale['Bilancio Energetico'],
                base=0,
                marker_color='yellow',
                name='Bilancio Energetico'
                ))

fig.show()

Grafico Torta Incentivo Condivisione

In [ ]:
parziale = df_finale[df_finale['Categoria'] != 'Produttore']
df = px.data.tips()
fig = px.pie(parziale, values=parziale['Incentivo Condivisione'], names=parziale['Identificativo'], color=parziale['Identificativo'])
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.0))
fig.show()

Grafico Torta Entrate della Comunità Energetica

In [ ]:
parziale = df_finale[ df_finale['Entrate Totali'] >0]
df = px.data.tips()
fig = px.pie(parziale, values=parziale['Entrate Totali'], names=parziale['Identificativo'], color=parziale['Identificativo'])
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.0))
fig.show()

Incentivo Condivisione VS Ritiro Dedicato (VALORE ECONOMICO)

In [ ]:
df = px.data.tips()
fig = go.Figure(data=[go.Pie(labels=['Incentivo Condivisione', 'Ritiro Dedicato'], values=[Incentivo_Totale_Condivisione, df_finale['Ritiro Dedicato'].sum()])])
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.24))
fig.show()

In [ ]:
fig = make_subplots(
    rows=5, cols=3,
    specs=[[{"rowspan": 1, "colspan": 3},None, None], 
          [{"rowspan": 1, "colspan": 3},None, None],  
          [{"rowspan": 1, "colspan": 3},None, None],  
          [{"rowspan": 1, "colspan": 3},None, None],
          [{"rowspan": 1, "colspan": 3},None, None]],  
          subplot_titles=("Incentivo Condivisione €", 'Costi Annuali CER Partecipanti €', "Ritiro Dedicato €", 'Beneficio Economico Totale €', 'Incidenza Percentuale su Bolletta %' ),
          shared_yaxes=True)

parziale = df_finale[df_finale['Identificativo'] != 'Produttore']

fig.add_trace(go.Bar(x=parziale['Identificativo'], y=parziale['Incentivo Condivisione'],
                     marker_color='blue', text=parziale['Incentivo Condivisione'].round(0)),
              1, 1)


fig.add_trace(go.Bar(x=df_finale['Identificativo'], y=df_finale['Costi Funzionamento CER'],
                     marker_color='purple', text=df_finale['Costi Funzionamento CER'].round(0)),
              2, 1)

fig.add_trace(go.Bar(x=df_finale['Identificativo'], y=df_finale['Ritiro Dedicato'],
                     marker_color='orange', text=df_finale['Ritiro Dedicato'].round(0)),
              3, 1)
fig.add_trace(go.Bar(x=parziale['Identificativo'], y=parziale['Beneficio Economico Totale'],
                     marker_color='cyan', text=parziale['Beneficio Economico Totale'].round(0)),
              4, 1)

fig.add_trace(go.Bar(x=df_finale[df_finale['Identificativo'] != 'Produttore']['Identificativo'], y=((df_finale['Beneficio Economico Totale']/df_finale['Costo Bolletta'])*100),
                     marker_color='red', text=((df_finale['Beneficio Economico Totale']/df_finale['Costo Bolletta'])*100).round(0)),
              5, 1)


fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.update_layout(barmode='stack')
fig.update_layout(height=2400, width=1200)
fig.update_traces(texttemplate= format('%{value:,.0f}'))
fig.show()